In [1]:
%pip install pyspark delta-spark pandas setuptools

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyspark
from delta import configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("aig") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

ss = configure_spark_with_delta_pip(builder).getOrCreate()

sql = ss.sql

25/02/09 13:03:16 WARN Utils: Your hostname, codespaces-f6f13d resolves to a loopback address: 127.0.0.1; using 10.0.2.145 instead (on interface eth0)
25/02/09 13:03:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13aab19a-bf21-49ae-8679-40c54c18e388;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 296ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3  

25/02/09 13:03:30 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
import subprocess

files = [
    'name.basics.tsv.gz',
    'title.akas.tsv.gz',
    'title.basics.tsv.gz',
    'title.crew.tsv.gz',
    'title.episode.tsv.gz',
    'title.principals.tsv.gz',
    'title.ratings.tsv.gz'
]

for file in files:
    subprocess.run(["wget", "https://datasets.imdbws.com/" + file, "-nc", "-O", "/tmp/" + file])    

for file in files:
    table_name = file.replace('.tsv.gz', '').replace('.', '_')
    ss.read.csv("/tmp/" + file, header=True, sep='\t', nullValue='\\N').createOrReplaceTempView(table_name + "_csv")

File ‘/tmp/name.basics.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.akas.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.basics.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.crew.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.episode.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.principals.tsv.gz’ already there; not retrieving.
File ‘/tmp/title.ratings.tsv.gz’ already there; not retrieving.


In [10]:
sql("""
SELECT * FROM title_basics_csv
""").show(truncate=False)

+---------+---------+-------------------------------------------+-------------------------------------------------+-------+---------+-------+--------------+------------------------+
|tconst   |titleType|primaryTitle                               |originalTitle                                    |isAdult|startYear|endYear|runtimeMinutes|genres                  |
+---------+---------+-------------------------------------------+-------------------------------------------------+-------+---------+-------+--------------+------------------------+
|tt0000001|short    |Carmencita                                 |Carmencita                                       |0      |1894     |NULL   |1             |Documentary,Short       |
|tt0000002|short    |Le clown et ses chiens                     |Le clown et ses chiens                           |0      |1892     |NULL   |5             |Animation,Short         |
|tt0000003|short    |Poor Pierrot                               |Pauvre Pierrot           